## *We have the same birthday!*

---

**Quick poll:** In a room of 23 people, what are the chances that two of them share the same birthday?

- A) About 5%
- B) About 25%
- C) About 50%
- D) About 75%

Most people guess A or B. The real answer is **C — over 50%**!

With just **70 people**, the probability jumps to **99.9%**.

Let's see why with math and simulation.

## The Math Behind It

It's easier to calculate the probability that **nobody** shares a birthday:

$$P(\text{no match with } n \text{ people}) = \frac{365}{365} \times \frac{364}{365} \times \frac{363}{365} \times \cdots \times \frac{365-n+1}{365}$$

Then:
$$P(\text{at least one match}) = 1 - P(\text{no match})$$

The key insight: we're counting **pairs**, not individuals. With 23 people there are $\binom{23}{2} = 253$ possible pairs!

In [1]:
import random
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import ipywidgets as widgets

sns.set_theme(style="ticks", font_scale=1.2)

In [2]:
def exact_birthday_probability(n: int) -> float:
    """Exact probability that at least 2 people in a group of n share a birthday."""
    if n > 365:
        return 1.0
    p_no_match = 1.0
    for k in range(n):
        p_no_match *= (365 - k) / 365
    return 1 - p_no_match


def simulate_birthday_probability(n: int, trials: int = 5000) -> float:
    """Monte Carlo estimate of the birthday match probability."""
    matches = 0
    for _ in range(trials):
        birthdays = [random.randint(1, 365) for _ in range(n)]
        if len(birthdays) != len(set(birthdays)):
            matches += 1
    return matches / trials


# Pre-compute the full curve (exact)
group_sizes  = list(range(1, 81))
exact_probs  = [exact_birthday_probability(n) for n in group_sizes]

Move the slider to see how the probability changes with group size.

In [ ]:
@widgets.interact(n=widgets.IntSlider(
    value=23, min=2, max=80, step=1,
    description="Group size:",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="500px"),
))
def plot_birthday(n):
    p_exact = exact_birthday_probability(n)
    pairs   = n * (n - 1) // 2

    fig, ax = plt.subplots(figsize=(9, 5))

    ax.plot(group_sizes, [p * 100 for p in exact_probs],
            color="#5B8FB9", linewidth=2.5, zorder=2, label="Exact probability")

    ax.fill_between(group_sizes[:n], [p * 100 for p in exact_probs[:n]],
                     alpha=0.15, color="#5B8FB9")

    ax.scatter([n], [p_exact * 100], color="#E8575A", s=120, zorder=5,
               label=f"n = {n}  →  {p_exact*100:.1f}%")

    ax.axhline(50, color="gray", linestyle="--", linewidth=1, label="50% threshold")

    ax.set_xlabel("Number of People in the Group", fontsize=12)
    ax.set_ylabel("Probability of a Shared Birthday (%)", fontsize=12)
    ax.set_title("Birthday Paradox — Probability vs. Group Size",
                 fontsize=14, weight="bold")
    ax.set_ylim(0, 105)
    ax.set_xlim(1, 80)
    ax.legend(fontsize=11)

    sns.despine()
    plt.tight_layout()
    plt.show()

    print(f"Group of {n} people")
    print(f"  Possible birthday pairs: {pairs:,}")
    print(f"  Probability of a shared birthday: {p_exact*100:.1f}%")
    if p_exact >= 0.5:
        print(f"  ✅ More likely than not!")
    else:
        print(f"  Still less than 50% — keep adding people!")

interactive(children=(IntSlider(value=23, description='Group size:', layout=Layout(width='500px'), max=80, min…

Let's verify the math by actually *running* the experiment thousands of times.

In [4]:
# Sample a few checkpoints and compare exact vs simulated
checkpoints = [10, 23, 30, 50, 70]

print(f"{'Group Size':>12} | {'Exact %':>10} | {'Simulated %':>12}")
print("-" * 40)
for n in checkpoints:
    exact = exact_birthday_probability(n) * 100
    sim   = simulate_birthday_probability(n, trials=10000) * 100
    print(f"{n:>12} | {exact:>9.1f}% | {sim:>11.1f}%")

  Group Size |    Exact % |  Simulated %
----------------------------------------
          10 |      11.7% |        12.3%
          23 |      50.7% |        50.9%
          30 |      70.6% |        70.9%
          50 |      97.0% |        97.3%
          70 |      99.9% |        99.9%


## Key Takeaways

1. **We're bad at estimating probabilities** — our brains think linearly, but probability compounds.
2. **Pairs grow much faster than people.** 23 people → 253 pairs. That's why 50% happens so early.
3. **Simulation confirms math.** When theory is hard to believe, we can verify it with code.

---
*Fun fact: I knew a classmate in my elementary class who shares the same birthday.*